In [ ]:
import openai
from openai import OpenAI
import pandas as pd

GPT_API_KEY = GPT_API_KEY

# Function to translate a single sentence
client = OpenAI(
  api_key=GPT_API_KEY,
)


def generate_gpt(prompt):
    completion = client.chat.completions.create(
      model="gpt-4-turbo-preview",
      response_format={ "type": "json_object" },
      messages=[
        {"role": "user", "content": prompt}
      ],
      temperature=0,
      logprobs=True,
    )

    return completion.choices[0].message.content, completion.choices[0].logprobs

In [ ]:
def get_prompt(question, response, ground_truth):
    Prompt = f"""
    ###Task Description:
    You will be provided with a potential answer along with a ground truth answer to a given question. Your task is to evaluate the potential answer if it’s matching the ground truth answer.
    Evaluation Steps:
    1. Write a detailed feedback that assess the matching of the response strictly based on the given score rubric, not evaluating in general.
    2. After writing a feedback, Assign a score for responses matching either 1 or 0, where 1 is the answer matches the ground-truth, wheras 0 when there is no match.
    3. The output format should look as follows: [RESULT] (either 1 or 0)\"
    4. Please do not generate any other opening, closing, and explanations.
    ###The instruction to evaluate:
    {question}
    ###Response to evaluate:
    {response}
    ###Reference Answer:
    {ground_truth}
    ###Score Rubrics:
    Score (0 or 1): “Response Match: Evaluate how much the evaluated answer matches the true answer.”
    ###Feedback (scores ONLY):
    the output should be in the json format like this :
    (Response Match: score 0 or 1)
    """
    return Prompt

In [ ]:
import json
data = {
        "question" : "What is the formula of chlorophyll",
        'response' : 'Chlorophyll is the pigmet used in photosynthesis, it helpes in generating oxygen. C55H72MgN4O5',
        'ground_truth' : 'The formula of chlorophyll is C55H72MgN4O5'
    }

prompt = get_prompt(data['question'], data['response'], data['ground_truth'])
feedback = generate_gpt(prompt)
print(feedback)


In [ ]:
def G_eval(data):
    scores = [compute_score(feedback) for feedback in [generate_gpt(get_prompt(data['question'], data['response'], data['ground_truth'])) for data in data]]
    return scores

In [ ]:
good_data = [
    {
        "question" : "What is the formula of chlorophyll",
        'response' : 'Chlorophyll a pigment usuall found in the leaves has the formula C55H72MgN4O5.',
        'ground_truth' : 'The formula of chlorophyll is C55H72MgN4O5'
    }
]

bad_data = [
    {
        "question" : "What is the formula of chlorophyll",
        'response' : 'Chlorophyll is the pigmet used in photosynthesis, it helpes in generating oxygen',
        'ground_truth' : 'The formula of chlorophyll is C55H72MgN4O5'
    }
]

data = good_data + bad_data
G_eval(data)